In [13]:
import pandas as pd
import numpy as np
import json
import requests
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
import urllib.request as req
import time
from urllib.request import urlopen
from scipy.stats import norm
import quandl


In [21]:
def get_historical_prices(symbol):
    site = 'https://api.iextrading.com/1.0/stock/' + symbol + '/chart/5y'
    data = json.loads(req.urlopen(site).read().decode())
    data = json.dumps(data)
    data = pd.read_json(data)
    data.index = pd.DatetimeIndex(data['date'])
    return data

def clean_options_df(df, spot_price, call_put_flag):
    df = df.drop(['contractSize', 'currency'], axis=1)
    df['expiration'] = pd.to_datetime(df['expiration'], unit='s')
    today = pd.to_datetime(pd.Timestamp.today().strftime('%D'))
    df['days to expiration'] = (df['expiration'] - today).dt.days
    df['B/A Spread'] = df['ask'] - df['bid']

    df = df.dropna()
    df = df.reset_index()
    df = df.drop('index', axis=1)
    df['lastTradeDate'] = pd.to_datetime(df['lastTradeDate'], unit='s')
    df['Days Since Last Trade'] = -(df['lastTradeDate'] - today).dt.days
    df['Spot Price'] = spot_price
    df['Call_Put_Flag'] = call_put_flag
    return df

In [22]:
def optionsdata(ticker):
    site = 'https://query2.finance.yahoo.com/v7/finance/options/' + ticker
    with urlopen(site) as response:
        DATA = response.read().decode()
        DATA = json.loads(DATA)

    Expirations = DATA["optionChain"]['result'][0]['expirationDates']
    Expiration_Dates = pd.to_datetime(Expirations, unit='s')
    Strikes = pd.Series(DATA["optionChain"]['result'][0]['strikes'])
    strikes = DATA["optionChain"]['result'][0]['strikes']
    ###collecting stock data####
    index = []
    entry = []
    for a in DATA["optionChain"]['result'][0]['quote'].items():
        index.append(a[0])
        entry.append(a[1])

    todays_quotes = pd.DataFrame(entry, index=index, columns=['Entry'])
    Master_List = []
    Master_Calls = []
    Master_Puts = []
    for expiration in Expirations:
        try:

            expiry = str(expiration)
            expiration_date = pd.to_datetime(expiration, unit='s')
            new_site = 'https://query2.finance.yahoo.com/v7/finance/options/' + ticker + '?date=' + expiry

            with urlopen(new_site) as response:
                DATA = response.read().decode()
                DATA = json.loads(DATA)
                Master_List.append(DATA)
                Call_df = pd.read_json(json.dumps(DATA['optionChain']['result'][0]['options'][0]['calls']))
                Put_df = pd.read_json(json.dumps(DATA['optionChain']['result'][0]['options'][0]['puts']))
                Master_Calls.append(Call_df)
                Master_Puts.append(Put_df)
        except:
            Exception

    Calls = pd.concat(Master_Calls)
    Puts = pd.concat(Master_Puts)
    spot = todays_quotes.loc['regularMarketPrice'].iloc[0]

    # Clean Call Data
    Calls = clean_options_df(Calls, spot, 'Call')
    Puts = clean_options_df(Puts, spot, 'Put')
    Calls[['delta', 'gamma', 'vega', 'theta', 'rho']] = Calls.apply(get_greeks_2, axis=1)
    Puts[['delta', 'gamma', 'vega', 'theta', 'rho']] = Puts.apply(get_greeks_2, axis=1)
    Calls[['$/Delta', '$/Gamma']] = Calls.apply(per_dollar_basis, axis=1)
    Puts[['$/Delta', '$/Gamma']] = Puts.apply(per_dollar_basis, axis=1)

    return Calls, Puts

def greeks(df):
    rate = 0.015
    q = 0
    spot = df['Spot Price']
    strike = df['strike']
    vol = df['impliedVolatility']
    maturity = df['days to expiration']
    Call_Put_Flag = df['Call_Put_Flag']

    """ask, bid, change, contractSymbol, expiration,vol, inTheMoney, lastPrice, lastTradeDate,
    openInterest, percentChange, strike, volume,maturity, BA_Spread, DaysSinceLastTrade,
    spot, Call_Put_Flag = list(column_list)"""
    d1 = (np.log(spot / strike) + (maturity / 365) * (rate - q + (vol ** 2) / 2)) / (vol * np.sqrt(maturity / 365))
    d2 = d1 - vol * np.sqrt(maturity / 365)
    gamma = ((np.exp(-q * maturity / 365) / (spot * vol * np.sqrt(maturity / 365))) * 1 / np.sqrt(2 * np.pi)) * np.exp(
        (-d1 ** 2) / 2)
    vega = (((spot * np.exp(-q * maturity / 365)) * np.sqrt(maturity / 365)) / 100) * 1 / np.sqrt(2 * 3.14159) * np.exp(
        (-d1 ** 2) / 2)

    if Call_Put_Flag == 'Call':
        try:
            delta = np.exp(-q * maturity / 365) * norm.cdf(d1)
        except:
            delta= np.nan
        try:
            theta = ((1 / (maturity / 365)) * -((spot * vol * np.exp(-q * maturity / 365))) / 2 * np.sqrt(
            maturity / 365)) * (np.exp((-d1 ** 2) / 2)) / np.sqrt(2 * 3.14159) - rate * strike * np.exp(
            -rate * maturity / 365) * norm.cdf(d2) + q * spot * np.exp(rate * maturity / 365) * norm.cdf(d1)
        except:
            theta = np.nan
        try:
            rho = strike * maturity / 365 * np.exp(-rate * maturity / 365) * norm.cdf(d2)
        except:
            rho = np.nan

    if Call_Put_Flag == 'Put':
        try:
            delta = np.exp(-rate * maturity / 365) * (norm.cdf(d1) - 1)
        except:
            delta = np.nan
        try:
            theta = ((1 / maturity / 365) * -(spot * vol * np.exp(-q * maturity / 365)) / 2 * np.sqrt(maturity / 365) * (
            np.exp((-d1 ** 2) / 2)) / np.sqrt(2 * 3.14159)) + rate * strike * np.exp(-rate * maturity / 365) * norm.cdf(
            -d2) - q * spot * np.exp(rate * maturity / 365) * norm.cdf(-d1)
        except:
            theta = np.nan
        try:

            rho = -strike * maturity / 365 * np.exp(-rate * maturity / 365) * norm.cdf(-d2)
        except:
            rho = np.nan

    return pd.Series([delta, gamma, vega, theta, rho])




In [24]:
calls,puts=optionsdata('AAPL')

In [25]:
calls

,ask,bid,change,contractSymbol,expiration,impliedVolatility,inTheMoney,lastPrice,lastTradeDate,openInterest,...,Days Since Last Trade,Spot Price,Call_Put_Flag,delta,gamma,vega,theta,rho,$/Delta,$/Gamma
0,81.15,80.55,0.000000,AAPL190222C00090000,2019-02-22,4.156255,True,80.95,2019-02-04 17:55:14,0,...,17,170.61,Call,0.998851,0.000103,0.000341,-27.242366,0.245994,81.043081,785914.293631
1,76.10,75.60,0.000000,AAPL190222C00095000,2019-02-22,3.839844,True,75.46,2019-02-15 19:54:07,7,...,6,170.61,Call,0.998710,0.000124,0.000380,-28.025102,0.259624,75.557454,608669.833416
2,71.10,70.60,0.000000,AAPL190222C00100000,2019-02-22,3.539064,True,70.64,2019-02-07 20:26:03,1,...,14,170.61,Call,0.998543,0.000150,0.000424,-28.903165,0.273243,70.743070,469831.927198
3,61.15,60.55,0.000000,AAPL190222C00110000,2019-02-22,2.974612,True,60.33,2019-02-15 16:15:06,11,...,6,170.61,Call,0.998116,0.000226,0.000536,-30.751888,0.300435,60.443852,266916.616588
4,56.05,55.60,1.090000,AAPL190222C00115000,2019-02-22,2.666019,True,56.65,2019-02-21 18:35:53,2,...,0,170.61,Call,0.998115,0.000252,0.000537,-27.830054,0.314137,56.757010,224437.859868
5,51.05,50.65,-1.199997,AAPL190222C00120000,2019-02-22,2.453129,True,51.65,2019-02-21 18:35:55,19,...,0,170.61,Call,0.997484,0.000356,0.000697,-32.996153,0.327531,51.780258,144975.619522
6,46.05,45.60,-1.669998,AAPL190222C00125000,2019-02-22,2.167973,True,46.40,2019-02-21 18:35:57,13,...,0,170.61,Call,0.997432,0.000411,0.000710,-29.966288,0.341209,46.519479,112960.605042
7,41.10,40.55,1.189999,AAPL190222C00130000,2019-02-22,1.931641,True,41.39,2019-02-21 18:35:59,16,...,0,170.61,Call,0.996925,0.000542,0.000835,-31.393172,0.354667,41.517672,76317.107827
8,39.10,38.60,0.000000,AAPL190222C00132000,2019-02-22,1.871094,True,35.13,2019-02-01 14:51:10,0,...,20,170.61,Call,0.996198,0.000677,0.001011,-36.486560,0.359797,35.264059,51858.478916
9,38.10,37.55,0.000000,AAPL190222C00133000,2019-02-22,1.792970,True,29.85,2019-01-30 17:50:17,0,...,22,170.61,Call,0.996543,0.000649,0.000928,-32.361939,0.362705,29.953564,45976.613650
